# MN Standard Deduction Reform - Excluding Single Filers

Testing reform without changing Single filer standard deduction to isolate the bug.

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

MN_DATASET = "hf://policyengine/policyengine-us-data/states/MN.h5"

In [2]:
# Reform WITHOUT changing Single filer standard deduction
def create_reform_no_single():
    reform = Reform.from_dict(
        {
            # SINGLE: NOT CHANGED (stays at $14,950)
            "gov.states.mn.tax.income.deductions.standard.base.SEPARATE": {
                "2025-01-01.2100-12-31": 20000
            },
            "gov.states.mn.tax.income.deductions.standard.base.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 30000
            },
            "gov.states.mn.tax.income.deductions.standard.base.JOINT": {
                "2025-01-01.2100-12-31": 40000
            },
            "gov.states.mn.tax.income.deductions.standard.base.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 40000
            },
        },
        country_id="us",
    )
    return reform

print("Reform (excluding Single filers):")
print("  - Single: $14,950 (UNCHANGED)")
print("  - Separate: $14,950 -> $20,000")
print("  - Head of Household: $22,500 -> $30,000")
print("  - Joint: $29,900 -> $40,000")
print("  - Surviving Spouse: $29,900 -> $40,000")

Reform (excluding Single filers):
  - Single: $14,950 (UNCHANGED)
  - Separate: $14,950 -> $20,000
  - Head of Household: $22,500 -> $30,000
  - Joint: $29,900 -> $40,000
  - Surviving Spouse: $29,900 -> $40,000


In [3]:
print("Loading simulations...")
baseline = Microsimulation(dataset=MN_DATASET)
reform_sim = Microsimulation(dataset=MN_DATASET, reform=create_reform_no_single())
print("Done!")

Loading simulations...
Done!


In [4]:
# Calculate impacts
tu_weight = np.array(baseline.calculate("tax_unit_weight", period=2025))
filing_status = np.array(baseline.calculate("filing_status", period=2025, map_to="tax_unit"))

baseline_income = np.array(baseline.calculate("household_net_income", period=2025, map_to="tax_unit"))
reform_income = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="tax_unit"))
income_change = reform_income - baseline_income

baseline_mn_tax = np.array(baseline.calculate("mn_income_tax", period=2025, map_to="tax_unit"))
reform_mn_tax = np.array(reform_sim.calculate("mn_income_tax", period=2025, map_to="tax_unit"))
mn_tax_change = reform_mn_tax - baseline_mn_tax

baseline_fed_tax = np.array(baseline.calculate("income_tax", period=2025, map_to="tax_unit"))
reform_fed_tax = np.array(reform_sim.calculate("income_tax", period=2025, map_to="tax_unit"))
fed_tax_change = reform_fed_tax - baseline_fed_tax

baseline_salt = np.array(baseline.calculate("salt_deduction", period=2025, map_to="tax_unit"))
reform_salt = np.array(reform_sim.calculate("salt_deduction", period=2025, map_to="tax_unit"))
salt_change = reform_salt - baseline_salt

tax_unit_itemizes = np.array(baseline.calculate("tax_unit_itemizes", period=2025, map_to="tax_unit"))

print("Data loaded!")

Data loaded!


In [5]:
# Overall results
losers_mask = income_change < -1
winners_mask = income_change > 1

print("="*70)
print("OVERALL RESULTS - REFORM EXCLUDING SINGLE FILERS")
print("="*70)

total_cost = (income_change * tu_weight).sum()
print(f"Net cost: ${total_cost/1e6:,.2f}M")
print(f"\nWinners: {tu_weight[winners_mask].sum():,.0f} ({tu_weight[winners_mask].sum()/tu_weight.sum()*100:.1f}%)")
print(f"Losers:  {tu_weight[losers_mask].sum():,.0f} ({tu_weight[losers_mask].sum()/tu_weight.sum()*100:.1f}%)")

if losers_mask.sum() > 0:
    print(f"\nAvg loss: ${np.average(income_change[losers_mask], weights=tu_weight[losers_mask]):,.2f}")

OVERALL RESULTS - REFORM EXCLUDING SINGLE FILERS
Net cost: $335.04M

Winners: 981,467 (46.2%)
Losers:  755 (0.0%)

Avg loss: $-65.65


In [6]:
# Losers by filing status
print("="*70)
print("LOSERS BY FILING STATUS")
print("="*70)

statuses = ["SINGLE", "JOINT", "SEPARATE", "HEAD_OF_HOUSEHOLD", "SURVIVING_SPOUSE"]

print(f"\n{'Status':<20} {'Losers':>10} {'Avg Loss':>12} {'Avg MN Tax':>12} {'Avg Fed Tax':>12} {'Avg SALT':>12}")
print("-"*80)

for status in statuses:
    mask = losers_mask & (filing_status == status)
    count = tu_weight[mask].sum()
    
    if count > 0:
        avg_loss = np.average(income_change[mask], weights=tu_weight[mask])
        avg_mn = np.average(mn_tax_change[mask], weights=tu_weight[mask])
        avg_fed = np.average(fed_tax_change[mask], weights=tu_weight[mask])
        avg_salt = np.average(salt_change[mask], weights=tu_weight[mask])
        print(f"{status:<20} {count:>10,.0f} ${avg_loss:>10,.0f} ${avg_mn:>10,.0f} ${avg_fed:>10,.0f} ${avg_salt:>10,.0f}")
    else:
        print(f"{status:<20} {0:>10} {'N/A':>12} {'N/A':>12} {'N/A':>12} {'N/A':>12}")

LOSERS BY FILING STATUS

Status                   Losers     Avg Loss   Avg MN Tax  Avg Fed Tax     Avg SALT
--------------------------------------------------------------------------------
SINGLE                      140 $       -18 $         0 $         0 $         0
JOINT                       613 $       -77 $        81 $         0 $         0
SEPARATE                      0          N/A          N/A          N/A          N/A
HEAD_OF_HOUSEHOLD             1 $       -40 $         0 $         0 $         0
SURVIVING_SPOUSE              0          N/A          N/A          N/A          N/A


In [7]:
# Key diagnostic: Single filers should have NO change
print("="*70)
print("SINGLE FILER CHECK (should have NO change)")
print("="*70)

single_mask = filing_status == "SINGLE"
single_count = tu_weight[single_mask].sum()

print(f"\nTotal Single filers: {single_count:,.0f}")

single_income_change = np.average(income_change[single_mask], weights=tu_weight[single_mask])
single_mn_change = np.average(mn_tax_change[single_mask], weights=tu_weight[single_mask])
single_fed_change = np.average(fed_tax_change[single_mask], weights=tu_weight[single_mask])
single_salt_change = np.average(salt_change[single_mask], weights=tu_weight[single_mask])

print(f"\nAvg income change: ${single_income_change:,.2f}")
print(f"Avg MN tax change: ${single_mn_change:,.2f}")
print(f"Avg Fed tax change: ${single_fed_change:,.2f}")
print(f"Avg SALT change: ${single_salt_change:,.2f}")

single_losers = tu_weight[single_mask & losers_mask].sum()
single_winners = tu_weight[single_mask & winners_mask].sum()

print(f"\nSingle filers winning: {single_winners:,.0f}")
print(f"Single filers losing: {single_losers:,.0f}")

if single_losers > 0 or single_winners > 0:
    print("\n⚠️  BUG: Single filers are affected even though their std deduction didn't change!")
else:
    print("\n✓ Single filers correctly unaffected")

SINGLE FILER CHECK (should have NO change)

Total Single filers: 1,144,853

Avg income change: $33.63
Avg MN tax change: $0.00
Avg Fed tax change: $0.00
Avg SALT change: $0.00

Single filers winning: 373,744
Single filers losing: 140

⚠️  BUG: Single filers are affected even though their std deduction didn't change!


In [8]:
# SALT vs MN Tax diagnostic for losers
print("="*70)
print("SALT vs MN TAX DIAGNOSTIC")
print("="*70)

if losers_mask.sum() > 0:
    loser_weights = tu_weight[losers_mask]
    
    avg_mn = np.average(mn_tax_change[losers_mask], weights=loser_weights)
    avg_salt = np.average(salt_change[losers_mask], weights=loser_weights)
    
    print(f"\nFor LOSERS:")
    print(f"  Avg MN tax change: ${avg_mn:,.2f}")
    print(f"  Avg SALT change:   ${avg_salt:,.2f}")
    print(f"  Difference:        ${avg_salt - avg_mn:,.2f}")
    
    # Zero MN change but losing
    zero_mn = losers_mask & (np.abs(mn_tax_change) < 1)
    if zero_mn.sum() > 0:
        print(f"\nTax units with MN tax ≈ $0 but losing: {tu_weight[zero_mn].sum():,.0f}")
        print(f"  Their avg SALT change: ${np.average(salt_change[zero_mn], weights=tu_weight[zero_mn]):,.2f}")
else:
    print("\nNo losers found!")

SALT vs MN TAX DIAGNOSTIC

For LOSERS:
  Avg MN tax change: $65.65
  Avg SALT change:   $0.00
  Difference:        $-65.65

Tax units with MN tax ≈ $0 but losing: 143
  Their avg SALT change: $0.00
